# Calibration
inspired by: http://jasss.soc.surrey.ac.uk/17/3/11.html

In [11]:
%matplotlib inline
import math
import numpy as np
import numpy.random as nrand
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import scipy.stats as stats
import powerlaw
from stockmarket import baselinemodel
import tqdm

# Helper functions

In [2]:
def hurst(price_series):
    """
    see:
    https://www.quantopian.com/posts/some-code-from-ernie-chans-new-book-implemented-in-python
    
    Function to calculate the hurst exponent
    """
    tau = []; lagvec = []  
    #  Step through the different lags  
    for lag in range(2,20):  
        #  produce price difference with lag  
        pp = np.subtract(price_series[lag:],price_series[:-lag])  
        #  Write the different lags into a vector  
        lagvec.append(lag)  
        #  Calculate the variance of the difference vector  
        tau.append(np.sqrt(np.std(pp)))  
    #  linear fit to double-log graph (gives power)  
    m = np.polyfit(np.log10(lagvec),np.log10(tau),1)  
    # calculate hurst  
    hurst = m[0]*2  
    # plot lag vs variance  
    #py.plot(lagvec,tau,'o'); show()  
    return hurst  

In [3]:
# return autocorrelation close to zero after lag 1
# calculate returns 
def calculate_returns(orderbook_transaction_price_history):
    """Return the returns"""
    averageprices = []
    for day in orderbook_transaction_price_history:
        averageprices.append(np.mean(day))
    returns = pd.Series(averageprices).diff()
    return returns[1:]

# Test 1
def zero_autocorrelation(returns, lags):
    """returns wether average autocorrelation is much different from zero"""
    autocorr_returns = [returns.autocorr(lag=lag) for lag in range(lags)]
    # if mean autocorrelation are between -0.1 and 0.1
    average_autocorrelation = np.mean(autocorr_returns[1:])
    if (average_autocorrelation < 0.1) and (average_autocorrelation > -0.1):
        return True
    else:
        return False

# Test 2 
def fat_tails(returns):
    results = powerlaw.Fit(returns)
    alpha = results.power_law.alpha
    if (alpha < 4) and (alpha > 2):
        return True
    else:
        return False
    
# Test 3
def clustered_volatility(returns, lags):
    absolute_returns = returns.abs()
    autocorr_abs_returns = [absolute_returns.autocorr(lag=lag) for lag in range(lags)]
    average_autocorrelation = np.mean(autocorr_abs_returns[1:])
    if (average_autocorrelation < 0.1) and (average_autocorrelation > -0.1):
        return False
    else:
        return True

# Test 4
def non_linear_dependence(returns, hurst_function):
    h = hurst_function(returns)
    if h > 0.5:
        return True
    else: 
        return False

In [15]:
# determine parameter space
speculators = {'exp1': 10, 'exp2': 30, 'exp3': 50, 'exp4': 80} 
# for the parameters
results = {}

for key, n_speculators in sorted(speculators.items()):
    #print (key, n_speculators)
    # simulate the model
    print ('now starting ', key)
    agents, firms, stocks, order_books, market_maker = baselinemodel.stockMarketSimulation(seed=0, simulation_time=250,
                                                                                       amount_momentum=n_speculators,
                                                                                       amount_mean_reversion=n_speculators,
                                                                                       amount_noise_traders=(200-2*n_speculators),
                                                                                       amount_firms=1,
                                                                                       initial_money=(100, 200),
                                                                                       initial_bid_ask=(1,1),
                                                                                       initial_memory=(3,7),
                                                                                       initial_ma_short=(20,40),
                                                                                       initial_ma_long=(120,150),
                                                                                       initial_profit=(200,200),
                                                                                       initial_book_value=(10000, 10000),
                                                                                       initial_stock_amount=4000,
                                                                                       order_expiration_time=150,
                                                                                       init_propensity_to_switch=1.1,
                                                                                       init_price_to_earnings_window=((4, 7), (10, 14)),
                                                                                       firm_profit_mu=0.058,
                                                                                       firm_profit_delta=0.00396825396,
                                                                                       firm_profit_sigma=0.125,
                                                                                       profit_announcement_working_days=20,
                                                                                       init_market_maker_money=5000,
                                                                                       market_maker_bid_ask_spread=3,
                                                                                       market_maker_price_to_earnings_window=(5,13),
                                                                                       market_maker_inventory_sensitivity=0.01,
                                                                                       market_maker_inventory_buffer_of_total_target=0.10,
                                                                                       m_m_standard_order_percentage_total=0.05,
                                                                                       agents_hold_thresholds=(0.9995, 1.0005),
                                                                                       init_share_of_stocks_to_market_maker=0.4,
                                                                                       init_backward_simulated_time=200,
                                                                                       trader_volume_risk_aversion=0.1,
                                                                                       printProgress=False
                                                                                       )
    # Apply the tests 
    # apply all tests to the model 
    # test 1
    first_test = zero_autocorrelation(calculate_returns(order_books[0].transaction_prices_history), 25)
    # test 2
    second_test = fat_tails(calculate_returns(order_books[0].transaction_prices_history))
    # test 3
    third_test = clustered_volatility(calculate_returns(order_books[0].transaction_prices_history), 25)
    # test 4
    fourth_test = non_linear_dependence(calculate_returns(order_books[0].transaction_prices_history), hurst)
    # save parameter space and test results
    results[key] = [first_test, second_test, third_test, fourth_test]

results

now starting  exp1
now starting  exp2
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume
no volume

Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


now starting 


Calculating best minimal value for power law fit

 exp3
now starting 


Calculating best minimal value for power law fit

 exp4


{'exp1': [True, False, False, False],
 'exp2': [True, True, True, False],
 'exp3': [True, True, True, False],
 'exp4': [True, False, False, False]}